# 以下函数均保存在DDS_CMpack.py中

# 天体力学第一次上机
我们需要完成以下任务：
1. 站心、地心、日心坐标系的转换 
2. 平太阳时和平恒星时的转换

> 时角和赤经方向相反

In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import datetime
import time

%matplotlib inline

In [6]:
def UTC2SG(yyyy,mm,dd,hours):
    # J2000，从UTC计算对应的jd_目的是最终算出儒略世纪数
    J2000 = 2451545.0
    # J2000：UTC时间2000年1月1日11:58:55.816。
    date_start = datetime.datetime.strptime("2000-1-1 11:58:55","%Y-%m-%d %H:%M:%S")
    Date_str = str(yyyy) +'-'+ str(mm) +'-'+ str(dd)
    Date = datetime.datetime.strptime(Date_str,"%Y-%m-%d")
    
    # 用unix时间戳计算到J2000的时间差
    un_start = time.mktime(date_start.timetuple()) + 0.816
    un_this = time.mktime(Date.timetuple()) + hours*3600
    
    # 返回儒略世纪数（1儒略年=31557600 SI秒）
    t = (un_this - un_start)/(31557600*100)
    
    # 返回以J2000开始的儒略日数量
    jd = t*(365.25*100)
    
    # 给出平恒星时(s为单位)
    SG = 18.6973746*3600 + 879000.0513367*3600*t + 0.093104*t**2 - 6.2e-6*t**3
    # degree = SG%(3600*24)/(3600*24)*360
    
    
    return SG,jd

In [7]:
# example，得出的SG去整除每天的秒数就可以得到当天的恒星时，最后再由恒星时算出相应的角度
SG,jd = UTC2SG(1999,3,22,6)
print(jd)
SG_degree = SG%(3600*24)/(3600*24)*360
SG_degree

-285.2492571296294


269.57287335637955

## 坐标系转换 - 坐标系平移
 从 A 到 B， 以下程序中 P_I 即 相对于I坐标系P的位置。同理 A_B 指的是在 B 坐标系中 A 的位置。P_B = A_B + P_A
 
 本次我们需要计算人造卫星以及小行星的初轨:
 * 对于人造卫星，我们需要用到 站心-地心 的转换。
 * 对于小行星，我们需要用到 站心-地心-日心，最后在日心：赤道-黄道
 
---

站心-地心坐标系转换时，我们可以先从GMT开始算起，SG即为格林尼治的恒星时 = 格林尼治春分点的时角 = 格林尼治相对于春分点的赤经

(格林尼治恒星时为4h时，春分点已经离开上中天4h，这时格林尼治相对于春分点逆时针走了4h，亦即它的赤经为4h）

steps:
* 格林尼治的赤经 + 地理经度 = 站台赤经； 地理纬度 = 站台赤纬；
* 站台赤经赤纬得到站台相对于地心的列向量 station_earth **其中需要一个赤经赤纬到x,y,z坐标系的转化**
* P_earth = P_station + station_earth

同理 地心-日心坐标系转换思路同上

* P_SolarBarycenter = P_earth + earth_SolarBarycenter 
 

In [4]:
# 赤经赤纬到x,y,z坐标系的转化:
def RADEC2xyz(r,RA,DEC):
    # input: RA,DEC in the unite of degree, r is the distance
    # steps: u(rad) = RA(deg); v(rad) = DEC(deg)
    # output: x,y,z 
    u,v = RA*2*np.pi/360,DEC*2*np.pi/360
    x = r*np.cos(u)*np.cos(v)
    y = r*np.sin(u)*np.cos(v)
    z = r*np.sin(v)
    return np.array([x,y,z])

In [5]:
# astronomy constants
r_p_station = 1
R_earth = 6371
au = 149597870
r_station_earth = 0.999102*R_earth
r_earth_sun = 1*au

# give the station's lam and phi
lam = 118.82091666
phi = 31.893611111

In [6]:
# give the station's RA and DEC then convert to station_earth
RA_station = SG_degree*(-1) + lam
DEC_station = phi
station_earth = RADEC2xyz(r_station_earth,RA_station,DEC_station)

# calculate the position of the object
alpha = 142.935
delta = 8.521111111
P_station = RADEC2xyz(r_p_station,alpha,delta)

#give the P_earth
P_earth = P_station + station_earth

print(P_earth)
print(P_station)  

## !!这个思路有大问题：矢量加减里头长度很重要！- 其实就是用这个去算

[-4716.12394061 -2639.90676817  3363.2029261 ]
[-0.78914393  0.59606743  0.14817381]


## 坐标系转换 - 赤道/黄道坐标系转换

黄道，赤道之间乘以一个旋转矩阵即可

Rx(-e)$*$黄道坐标 = 赤道坐标

Rx(e)$*$赤道坐标 = 黄道坐标

In [23]:
def Rx(x):
    import numpy as np
    # x is in the unit of deg, while xx is in the unit of rad
    xx = x*2*np.pi/360
    Rx = np.array([[1,0,0],\
                     [0,np.cos(xx),np.sin(xx)],\
                     [0,-np.sin(xx),np.cos(xx)]])
    return Rx

def equator2ecliptic(origin,epsilon,key):
    # epsilon是黄赤交角
    # from equ to ecl , key = 1
    # from ec1 to equ , key = 2
    # origin是个np.的1x3向量
    if key == 1:
        output = np.dot(origin,Rx(epsilon).T)
    elif key == 2:
        output = np.dot(origin,Rx(-epsilon).T)
    else:
        print('you need to input a key as: \n from equ to ecl , key = 1' + \
        '\n from ec1 to equ , key = 2')
    
    return output

In [22]:
np.array([[1,2,3],[4,5,6],[7,8,9]]).T

array([[1, 4, 7],
       [2, 5, 8],
       [3, 6, 9]])

In [17]:
print(np.array([1,2,3]))

[1 2 3]
